In [57]:
from IPython.display import display, HTML
import pandas as pd

Grab the anomaly tables from the wiki

In [23]:
anomaly_url = 'https://stellaris.paradoxwikis.com/Anomaly'
tables = pd.read_html(anomaly_url)
f'Found {len(tables)} tables'

'Found 20 tables'

Unfortunately, not all table defintions are actually useful. Inspect tables to find which and modify below if needed.

In [24]:
useful_table_indices = [0, 2, 4]

Merge the tables under common column names.

In [93]:
dfs = []
for i in useful_table_indices:
    table = tables[i]
    description = 'Reward' if 'Reward' in table.columns else 'Possible outcomes'
    dfs.append(
        pd.DataFrame({
            'anomaly': table['Anomaly'],  
            'description': table[description] + ' ' + table['Celestial body']
        })
    )
df = pd.concat(dfs)

Pair key words in the anomaly description. Order matters such that the first category to be found is the chosen category.

In [158]:
categories = dict(
    red=['scientist dies'],
    green=[
        'Any Habitable Planet', 'technology', 'minerals',
        'influence', 'unity', 'corvettes'
    ],
    yellow=[
        'scaled', 'deposit', 'archaeological', 'L-Gate',
        'Physics Research', 'Society Research', 'Engineering Research'],
)

In [159]:
def like_function(x):
    group = 'white'
    for key, values in categories.items():
        for value in values:
            if value.lower() in x.lower():
                return key
    return group

df['category'] = df['description'].apply(like_function)

In [160]:
df['category'].value_counts()

category
green     83
yellow    63
white     19
red        1
Name: count, dtype: int64

In [162]:
# display(HTML(df[df['category'] == 'white'].to_html()))